# 01 Attribute and Pickle the Network
Step 01: Add key attributes to the 'raw' network and create pickles

In [1]:
import os
import sys
import pickle

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O

In [4]:
input_dir = os.path.join("D:/metcouncil_network_rebuild/data/processed/version_01")
lasso_dir = os.path.join("Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities")
output_dir = input_dir

## Read Roadway and Transit Networks

In [5]:
link_file = os.path.join(input_dir, 'standard_networks', 'links.json')
node_file = os.path.join(input_dir, 'standard_networks', 'nodes.geojson')
shape_file = os.path.join(input_dir, 'standard_networks', 'shapes.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2022-03-21 18:29:33, INFO: Reading from following files:
-D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\links.json
-D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\nodes.geojson
-D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\shapes.geojson.
2022-03-21 18:29:33, INFO: Reading from following files:
-D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\links.json
-D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\nodes.geojson
-D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\shapes.geojson.
2022-03-21 18:32:08, INFO: Read 1056575 links from D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\links.json
2022-03-21 18:32:08, INFO: Read 1056575 links from D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\links.json
2022-03-21 18:32:08, INFO: Read 412227 nodes from D:/metcouncil_network_rebuild/da

In [6]:
roadway_net.links_df.shape

(1056575, 41)

In [7]:
roadway_net.links_df.model_link_id.nunique()

1056575

In [8]:
roadway_net.links_df.shstReferenceId.nunique()

1056575

In [9]:
print(roadway_net.links_df.u.isnull().sum())
print(roadway_net.links_df.u.nunique())

0
412131


In [10]:
roadway_net.links_df[roadway_net.links_df.A < 3101]

,shstReferenceId,id,shstGeometryId,fromIntersectionId,toIntersectionId,u,v,nodeIds,wayId,roadClass,...,walk_access,bike_access,county,length,model_link_id,A,B,locationReferences,rail_only,geometry


In [11]:
roadway_net.links_df.iloc[0]

shstReferenceId                        000030c9843ec7a7076fbcbbb0b79957
id                                     c452f95293d9bda3e4788c9ef5843a59
shstGeometryId                         c452f95293d9bda3e4788c9ef5843a59
fromIntersectionId                     2c7945f906f016bb8248964cd14f597a
toIntersectionId                       4698fe306022179796ea0bb24b76a22a
u                                                             187058420
v                                                             187070777
nodeIds                                          [187058420, 187070777]
wayId                                                          18102901
roadClass                                                       Service
oneWay                                                            False
roundabout                                                        False
link                                                              False
oneway                                                          

In [12]:
roadway_net.links_df.v.isnull().sum()

0

In [13]:
roadway_net.nodes_df.shape

(412227, 11)

In [14]:
roadway_net.nodes_df.model_node_id.nunique()

412227

In [15]:
roadway_net.nodes_df.shst_node_id.nunique()

412227

In [16]:
roadway_net.nodes_df.osm_node_id.nunique()

412227

In [17]:
roadway_net.nodes_df.columns

Index(['osm_node_id', 'shst_node_id', 'county', 'drive_access', 'walk_access',
       'bike_access', 'model_node_id', 'rail_only', 'geometry', 'X', 'Y'],
      dtype='object')

In [18]:
roadway_net.links_df.columns

Index(['shstReferenceId', 'id', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'u', 'v', 'nodeIds', 'wayId', 'roadClass', 'oneWay',
       'roundabout', 'link', 'oneway', 'lanes', 'ref', 'name', 'highway',
       'service', 'width', 'maxspeed', 'access', 'junction', 'bridge',
       'tunnel', 'landuse', 'area', 'key', 'forward', 'roadway',
       'drive_access', 'walk_access', 'bike_access', 'county', 'length',
       'model_link_id', 'A', 'B', 'locationReferences', 'rail_only',
       'geometry'],
      dtype='object')

In [19]:
transit_net = TransitNetwork.read(feed_path = os.path.join(input_dir, 'standard_networks', 'transit'))

2022-03-21 18:33:30, INFO: Read in transit feed from: D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\transit
2022-03-21 18:33:30, INFO: Read in transit feed from: D:/metcouncil_network_rebuild/data/processed/version_01\standard_networks\transit
2022-03-21 18:33:30, INFO: Removing calendar.txt from transit network config because file not found
2022-03-21 18:33:30, INFO: Removing calendar.txt from transit network config because file not found
2022-03-21 18:33:30, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-03-21 18:33:30, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-03-21 18:33:31, INFO: Removing fare_attributes.txt from transit network config because file not found
2022-03-21 18:33:31, INFO: Removing fare_attributes.txt from transit network config because file not found
2022-03-21 18:33:31, INFO: Removing fare_rules.txt from transit network config because file not found


## Keep the Drive Links
During the network updates, this allows for faster I/O. Also remove the legacy number of lanes coding, which is now computed via the `calculate_number_of_lanes` method.

In [20]:
# roadway_net.links_df = roadway_net.links_df[roadway_net.links_df['drive_access'] == 1]
roadway_net.links_df = roadway_net.links_df.drop('lanes', axis = 1)

## Attribute the Network

In [21]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2022-03-21 18:33:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-03-21 18:33:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


In [22]:
roadway_net.links_df.drive_access.value_counts()

True     652933
False    403642
Name: drive_access, dtype: int64

In [23]:
r_net = metcouncil.calculate_number_of_lanes_from_reviewed_network(
    roadway_net=roadway_net,
    parameters=parameters,
)
r_net.links_df.lanes.value_counts()

2022-03-21 18:33:34, INFO: Calculating Number of Lanes as network variable: 'lanes'
2022-03-21 18:33:34, INFO: Calculating Number of Lanes as network variable: 'lanes'
2022-03-21 18:33:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-03-21 18:33:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-03-21 18:33:34, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2022-03-21 18:33:34, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2022-03-21 18:33:34, INFO: Finished calculating centroid connector variable: centroidconnect
2022-03-21 18:33:34, INFO: Finished calculating centroid connector variable: centroidconnect
2022-03-21 18:34:32, INFO: Finished calculating number of lanes to: lanes
2022-03-21 18:34:32, INFO: Finished calculating number of lanes to: lanes


1.0    1021569
2.0      31969
3.0       2591
4.0        369
5.0         71
6.0          5
7.0          1
Name: lanes, dtype: int64

In [24]:
r_net.links_df.iloc[0]

shstReferenceId                        000030c9843ec7a7076fbcbbb0b79957
id                                     c452f95293d9bda3e4788c9ef5843a59
shstGeometryId                         c452f95293d9bda3e4788c9ef5843a59
fromIntersectionId                     2c7945f906f016bb8248964cd14f597a
toIntersectionId                       4698fe306022179796ea0bb24b76a22a
u                                                             187058420
v                                                             187070777
nodeIds                                          [187058420, 187070777]
wayId                                                          18102901
roadClass                                                       Service
oneWay                                                            False
roundabout                                                        False
link                                                              False
oneway                                                          

In [25]:
r_net.links_df['bus_only'] = 0

In [26]:
r_net = metcouncil.calculate_assign_group_and_roadway_class_from_reviewed_network(
        roadway_net=r_net,
        parameters=parameters,
)
r_net.links_df.assign_group.value_counts()

2022-03-21 18:34:33, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2022-03-21 18:34:33, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2022-03-21 18:34:33, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2022-03-21 18:34:33, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2022-03-21 18:36:31, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class
2022-03-21 18:36:31, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class


101.0    403642
50.0     291998
7.0      101402
6.0       76353
5.0       14986
15.0       3045
3.0        2590
1.0        2450
4.0        2031
2.0        1256
11.0        153
Name: assign_group, dtype: int64

In [27]:
r_net.links_df.roadway_class.value_counts()

101.0    403642
50.0     301921
40.0     124531
30.0      42088
20.0      20421
60.0       5411
10.0       1773
70.0        119
Name: roadway_class, dtype: int64

In [28]:
r_net.links_df.iloc[0]

shstReferenceId                        000030c9843ec7a7076fbcbbb0b79957
id                                     c452f95293d9bda3e4788c9ef5843a59
shstGeometryId                         c452f95293d9bda3e4788c9ef5843a59
fromIntersectionId                     2c7945f906f016bb8248964cd14f597a
toIntersectionId                       4698fe306022179796ea0bb24b76a22a
u                                                             187058420
v                                                             187070777
nodeIds                                          [187058420, 187070777]
wayId                                                          18102901
roadClass                                                       Service
oneWay                                                            False
roundabout                                                        False
link                                                              False
oneway                                                          

### add rail links and nodes

In [29]:
r_net = metcouncil.add_rail_links_and_nodes(
    roadway_network = r_net,
    parameters = parameters,
    rail_links_file = os.path.join(input_dir, 'standard_networks', 'rail_links.geojson'),
    rail_nodes_file = os.path.join(input_dir, 'standard_networks', 'rail_nodes.geojson'),
)

2022-03-21 18:36:32, INFO: Adding centroid and centroid connector to standard network
2022-03-21 18:36:32, INFO: Adding centroid and centroid connector to standard network
2022-03-21 18:36:32, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-03-21 18:36:32, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
1056575
  agency_raw_name shape_id from_stop_id to_stop_id  rail_only  \
0          MT2019  8880001        53342      53343          1   
1          MT2019  8880001        53343      53344          1   
2          MT2019  8880001        53344      53345          1   

  fromIntersectionId toIntersectionId  \
0       MT2019_53342     MT2019_53343   
1       MT2019_53343     MT2019_53344   
2       MT2019_53344     MT2019_53345   

                                            geometry  lanes  assign_group  \
0  LINESTRING (-93.27701 44.98332, -93.27647 44.9..

In [30]:
# check if missing IDs
# rail nodes does not have osm IDs, they have shst IDs
# rail links does not have osm IDs, they have shst IDs

# if node missing shst id
print(r_net.nodes_df.shst_node_id.isnull().sum())
print(r_net.nodes_df.shst_node_id.nunique())
# if node missing osm id
print(r_net.nodes_df.osm_node_id.isnull().sum())
# if node missing osm id
print(r_net.nodes_df.osm_node_id.dtype)
print(r_net.nodes_df[r_net.nodes_df.osm_node_id == 0])
print(r_net.nodes_df.osm_node_id.nunique())
# if node missing model node id
print(r_net.nodes_df.model_node_id.nunique())

# if link missing 
print(r_net.links_df.shstReferenceId.isnull().sum())
print(r_net.links_df.shstReferenceId.nunique())
print(r_net.links_df.model_link_id.nunique())

# if link missing node id
print(r_net.links_df.fromIntersectionId.isnull().sum())
print(r_net.links_df.toIntersectionId.isnull().sum())
print(r_net.links_df.u.isnull().sum())
print(r_net.links_df[r_net.links_df.u == 0])
print(r_net.links_df.v.isnull().sum())
print(r_net.links_df[r_net.links_df.v == 0])

0
412318
91
float64
Empty GeoDataFrame
Columns: [osm_node_id, shst_node_id, county, drive_access, walk_access, bike_access, model_node_id, rail_only, geometry, X, Y]
Index: []
412227
412318
0
1056665
1056665
0
0
90
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, ref, name, highway, service, width, maxspeed, access, junction, bridge, tunnel, landuse, area, key, forward, roadway, drive_access, walk_access, bike_access, county, length, model_link_id, A, B, locationReferences, rail_only, geometry, centroidconnect, lanes, bus_only, assign_group, roadway_class]
Index: []

[0 rows x 45 columns]
90
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, ref, name, highway, service, width, maxspeed, access, junction, bridge, tunnel, landuse, area, key, forward,

## Create a Scenario

In [31]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}

In [32]:
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2022-03-21 18:36:38, INFO: Creating Scenario
2022-03-21 18:36:38, INFO: Creating Scenario


## Make Pickles

In [33]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_00.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))